Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [3]:
ratings.head()
# ratings.info()
ratings.rating.value_counts()

4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: rating, dtype: int64

In [4]:
%%time

tuple_classification = {0.5 : 'низкий рейтинг', 
                        1.0 : 'низкий рейтинг', 
                        1.5 : 'низкий рейтинг',
                        2.0 : 'низкий рейтинг',
                        2.5 : 'средний рейтинг',
                        3.0 : 'средний рейтинг',
                        3.5 : 'средний рейтинг',
                        4.0 : 'средний рейтинг',
                        4.5 : 'высокий рейтинг',
                        5.0 : 'высокий рейтинг'}

def movie_classification(row):
    return tuple_classification[row]
        

ratings['classification'] = ratings['rating'].apply(movie_classification)
ratings

Wall time: 35.9 ms


,userId,movieId,rating,timestamp,classification
0,1,1,4.0,964982703,средний рейтинг
1,1,3,4.0,964981247,средний рейтинг
2,1,6,4.0,964982224,средний рейтинг
3,1,47,5.0,964983815,высокий рейтинг
4,1,50,5.0,964982931,высокий рейтинг
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,средний рейтинг
100832,610,168248,5.0,1493850091,высокий рейтинг
100833,610,168250,5.0,1494273047,высокий рейтинг
100834,610,168252,5.0,1493846352,высокий рейтинг


Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [5]:
keywords = pd.read_csv('keywords.csv')
keywords.head(100)
# keywords.info()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195
...,...,...
95,учи ру,1335038
96,яндекс главная страница,1334744
97,однокласники,1320152
98,мтс личный кабинет,1766893


In [6]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

# создадим словарь {'ГОРОД':'Регион'}
cityes = dict()
for region, city_list in geo_data.items():
    for city in city_list:
        cityes.setdefault(city.upper(), region)



In [7]:
def geo(words):
    words = words.upper()
    for word in words.split(' '):
        if word in cityes:
            return cityes[word]
    return 'undefined'

keywords['REGION'] = keywords['keyword'].apply(geo)

In [8]:
keywords['REGION'].value_counts()

undefined         99406
Центр               384
Северо-Запад        146
Дальний Восток       64
Name: REGION, dtype: int64

Задание 3 (бонусное)
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [15]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')

In [16]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [17]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [18]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [154]:
# взял все года для интереса
years = list()
for i in range(1900, 2020):
    years.append(str(i))

In [155]:
# import re
# def production_year(row):
#     regex = r'(\()(\d{4})(\))'
#     pattern = re.search(regex, row)
#     if pattern and (pattern.group(2) in years):
#         return pattern.group(2)
#     return '1900'


# print(production_year('The Fate of the Furious (2017)'))

In [156]:
def production_year(row):
    for year in years:
        if str(year) in row:
            return str(year)
    return '1900'

print(production_year('924,2001: A Space Odyssey (1968),Adventure|Drama|Sci-Fi'))

1968


In [157]:
joined['year'] = joined['title'].apply(production_year)
joined[joined['year'] == '1900']

,userId,movieId,rating,timestamp,title,genres,year
9147,62,176601,5.0,1525795252,Black Mirror,(no genres listed),1900
16886,105,147250,5.0,1526207354,The Adventures of Sherlock Holmes and Doctor W...,(no genres listed),1900
17879,111,171631,1.0,1517440909,Maria Bamford: Old Baby,(no genres listed),1900
17881,111,171891,3.5,1517440199,Generation Iron 2,(no genres listed),1900
26514,182,2691,4.5,1075801143,"Legend of 1900, The (a.k.a. The Legend of the ...",Drama,1900
30039,209,140956,3.5,1524522239,Ready Player One,Action|Sci-Fi|Thriller,1900
30090,210,40697,4.0,1527266191,Babylon 5,Sci-Fi,1900
46593,305,140956,4.5,1533043577,Ready Player One,Action|Sci-Fi|Thriller,1900
46711,306,149334,3.0,1518380372,Nocturnal Animals,Drama|Thriller,1900
49851,318,171495,5.0,1516826406,Cosmos,(no genres listed),1900


In [158]:
top = joined.groupby(joined.year).mean().sort_values('rating', ascending=False)['rating'].head(50)
top.head()

year
1917    4.500000
1930    4.205882
1921    4.100000
1934    4.088235
1944    4.043478
Name: rating, dtype: float64

In [159]:


top_decade = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#index        00 01 02 03 04 05 06 07 08 09 10 11 12
#decade       00 10 20 30 40 50 60 70 80 90 00 10 20
for key, data in top.items():
    top_decade[int(str(int(key[0]) - 1) + key[2])] += data

list_sort = sorted((e, i) for i,e in enumerate(top_decade))    

for data in list_sort[::-1]:
    print(f'{1900 + data[1] * 10} - {1910 + data[1] * 10} г.г. - суммарный средний рейтинг =  {data[0]:.1f}')
    

1950 - 1960 г.г. - суммарный средний рейтинг =  34.7
1960 - 1970 г.г. - суммарный средний рейтинг =  34.3
1940 - 1950 г.г. - суммарный средний рейтинг =  31.1
1970 - 1980 г.г. - суммарный средний рейтинг =  30.7
1930 - 1940 г.г. - суммарный средний рейтинг =  23.5
1920 - 1930 г.г. - суммарный средний рейтинг =  23.2
1980 - 1990 г.г. - суммарный средний рейтинг =  7.5
1910 - 1920 г.г. - суммарный средний рейтинг =  4.5
1900 - 1910 г.г. - суммарный средний рейтинг =  4.0
2020 - 2030 г.г. - суммарный средний рейтинг =  0.0
2010 - 2020 г.г. - суммарный средний рейтинг =  0.0
2000 - 2010 г.г. - суммарный средний рейтинг =  0.0
1990 - 2000 г.г. - суммарный средний рейтинг =  0.0


In [160]:
top_data = pd.DataFrame(top_decade)
# top_data.info()
print(f'{top_data.sort_values(0, ascending=False)}')

            0
5   34.659513
6   34.306900
4   31.070304
7   30.681062
3   23.523817
2   23.224962
8    7.494313
1    4.500000
0    4.000000
9    0.000000
10   0.000000
11   0.000000
12   0.000000
